# About the Dataset
The dataset we are going to use is of Detecting Hate Speech in people's tweets.

In [6]:
# Load dataset
import pandas as pd

dataset = pd.read_csv('basicmlmodel.csv', sep = ',')
dataset.head()

id  label                                              tweet
0   1      0   @user when a father is dysfunctional and is s...
1   2      0  @user @user thanks for #lyft credit i can't us...
2   3      0                                bihday your majesty
3   4      0  #model   i love u take with u all the time in ...
4   5      0             factsguide: society now    #motivation

# Key Note
label----> is the column that contains the target variable or the that has to be predicgted.

            1 -- hate speech
            0 -- Not hate speech
    
tweet-----> is the column that contains the text of the tweet. This column is the main data on which we can appled
            NLP techniques

In [7]:
# Let;s have close look at some of tweets
for index, tweet in enumerate(dataset['tweet'][10:15]):
    print(index + 1, ".", tweet)

1 .  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 . we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 . i get to see my daddy today!!   #80days #gettingfed
4 . ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 . i am thankful for having a paner. #thankful #positive     


We have seen in tweets there are alot of noisy data, like symbols, hash tags,
unicode character's etc are all meaningless data.

So we need Data Cleaning

# Data Cleanup

to overcome noise from text

In [10]:
import re

def clean_text(text):
    #Filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    #convert to lowercase to maintain consistancy
    text = text.lower()
    
    return text

dataset['clean_text'] = dataset.tweet.apply(lambda x: clean_text(x))
dataset.head()


id  label                                              tweet  \
0   1      0   @user when a father is dysfunctional and is s...   
1   2      0  @user @user thanks for #lyft credit i can't us...   
2   3      0                                bihday your majesty   
3   4      0  #model   i love u take with u all the time in ...   
4   5      0             factsguide: society now    #motivation   

                                          clean_text  
0    user when a father is dysfunctional and is s...  
1   user  user thanks for  lyft credit i can't us...  
2                                bihday your majesty  
3   model   i love u take with u all the time in ...  
4             factsguide  society now     motivation

# Feature Engineering

In [11]:
#list of stopwords in the english language.

from wordcloud import STOPWORDS
print(STOPWORDS)

{'these', 'him', 'further', "i'm", 'do', 'did', 'which', 'and', 'his', "didn't", 'hence', 'before', 'your', 'after', 'our', 'by', 'is', "she's", 'only', 'if', "it's", 'same', 'have', "he's", 'up', 'what', "they're", "who's", "couldn't", 'could', "i'll", 'should', 'shall', "shan't", 'against', "they've", 'were', 'those', 'this', 'above', 'for', 'get', 'himself', 'nor', 'into', "he'd", 'until', "haven't", 'been', 'cannot', 'since', 'from', 'own', 'about', 'few', 'while', 'during', 'very', 'itself', 'but', 'as', 'or', 'yours', "here's", 'just', 'ours', "mustn't", 'that', 'my', 'out', 'a', 'once', 'more', 'yourselves', 'had', 'am', "hadn't", 'no', 'k', 'can', 'themselves', "we're", 'off', 'they', 'each', 'an', 'here', 'again', 'being', 'theirs', 'who', 'too', "where's", 'at', 'her', 'of', 'ought', "she'd", 'when', 'how', 'so', 'are', "what's", 'it', 'we', "doesn't", "you're", 'than', 'in', 'any', "you'll", 'most', "we'll", 'com', 'the', 'you', "wouldn't", 'however', 'does', 'me', "we'd", "

In [18]:
#generate word frequency
def gen_freq(text):
    #Will store the list of words
    word_list = []
    
    #loop over all the tweets and extract words into word_list
    for tw_words in text.split():
        word_list.extend(tw_words)
        
    #Create word frequencies using word_list
    word_freq = pd.Series(word_list).value_counts()
    
    #Drop the stopwords during the frequency calculation
    word_freq = word_freq.drop(STOPWORDS, errors='ignore')
    
    return word_freq

In [19]:
#check weather a negative term is present in the text
def any_neg(words):
    for word in words:
        if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
            return 1
    else:
        return 0

In [24]:
#check weather one of 100 rare words is present in the text
def any_rare(words, rare_100):
    for word in words:
        if word in rare_100:
            return 1
    else:
        return 0

In [25]:
#check weather prompt words are present
def is_question(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who']:
            return 1
    else:
        return 0

In [26]:
word_freq = gen_freq(dataset.clean_text.str)

#100 most rare words in the dataset
rare_100 = word_freq[-100:]

#Number of words in a tweet
dataset['word_count'] = dataset.clean_text.str.split().apply(lambda x: len(x))

#Negative words are present or not
dataset['any_neg'] = dataset.clean_text.str.split().apply(lambda x: any_neg(x))

#prompt present or not
dataset['is_question'] = dataset.clean_text.str.split().apply(lambda x: is_question(x))

#Any of the most 100 rare words present or not
dataset['any_rare'] = dataset.clean_text.str.split().apply(lambda x: any_rare(x, rare_100))

#character count of the tweet
dataset['char_count'] = dataset.clean_text.apply(lambda x: len(x))

In [27]:
#Top 10 common words are 
gen_freq(dataset.clean_text.str)[:10]

user      3351
amp        439
love       320
day        254
trump      214
happy      207
will       191
people     186
new        171
u          158
dtype: int64

In [28]:
dataset.head()

id  label                                              tweet  \
0   1      0   @user when a father is dysfunctional and is s...   
1   2      0  @user @user thanks for #lyft credit i can't us...   
2   3      0                                bihday your majesty   
3   4      0  #model   i love u take with u all the time in ...   
4   5      0             factsguide: society now    #motivation   

                                          clean_text  word_count  any_neg  \
0    user when a father is dysfunctional and is s...          18        0   
1   user  user thanks for  lyft credit i can't us...          19        1   
2                                bihday your majesty           3        0   
3   model   i love u take with u all the time in ...          12        0   
4             factsguide  society now     motivation           4        0   

   is_question  any_rare  char_count  
0            1         0         102  
1            0         0         122  
2            0         0          21  
3            0         0          86  
4            0         0          39

# Spliting the dataset for trainning & testing

In [33]:
from sklearn.model_selection import train_test_split

X = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
y = dataset.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,random_state = 30)

# Train ML model for Text Classification

In [34]:
# we are using Naive Bays classifier
from sklearn.naive_bayes import GaussianNB

#Intialize Classifier
model = GaussianNB()

# fit the model
model = model.fit(X_train, y_train)

#make prediction on test data set

pred = model.predict(X_test)

# Evaluate the model

In [35]:
from sklearn.metrics import accuracy_score
print("Accuracy:" , accuracy_score(y_test, pred)*100, "%")

Accuracy: 57.714285714285715 %
